In [ ]:
# default_exp torch_dataset

# Torch dataset

Now that you have compose your dataset you would like to be able to use it to train a neural network for example.
You may want a torch dataset:



In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from typing import Callable

from docarray import Document, DocumentArray
from torch.utils.data import Dataset
from torchvision.datasets.folder import default_loader
import torchvision as tv

In [ ]:
# export
class TorchDataset(Dataset):
    """
    A torch dataset for web2dataset  that you can use to train model with pytorch
    """

    def __init__(self, path: str ,transform : Callable = tv.transforms.ToTensor()):
        """
        path: folder from where to load the web2dataset dataset
        transform: a torchvision transform function to apply augmentation and converstion
        """
        self.path = path
        self.transform = transform

        with open(f"{path}/dataset.bin", "rb") as f:
            self.docs = DocumentArray.from_bytes(f.read())

    def __getitem__(self, key):

        document = self.docs[key]
        tensor = default_loader(f"{self.path}/{document.uri}")
        tensor = self.transform(tensor)

        return tensor, document.tags

    def __len__(self):
        return len(self.docs)

## Example

In [ ]:
test_folder = "/tmp/test"

In [ ]:
# hide
import shutil

shutil.rmtree(test_folder, ignore_errors=True)

In [ ]:
from web2dataset.downloader import GoogleImageDownloader

downloader = GoogleImageDownloader(f"{test_folder}/bikedataset")
downloader.download("gravel bike", 16)

Output()

In [ ]:
!ls /tmp/test/bikedataset

dataset.bin  images


In [ ]:
dataset = TorchDataset(f"{test_folder}/bikedataset")

In [ ]:
dataset[0][0].shape, dataset[0][1]

(torch.Size([3, 168, 300]),
 {'tag': {'uuid': '05efe344-bcbf-11ec-8530-645d865124e9',
   'origin': 'gravel bike'}})